In [1]:
import numpy as np
import h5py
import os
import random
from keras import optimizers

import tensorflow as tf
from keras.layers import Input, Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, Conv1D, MaxPooling1D
from keras.layers.convolutional import ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint

from keras.initializers import glorot_uniform
import sklearn
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd

from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.utils import shuffle

SEED_VAL = 128
os.environ['SEED_NN'] = str(SEED_VAL)
random.seed(SEED_VAL)
np.random.seed(SEED_VAL)
tf.set_random_seed(SEED_VAL)


Using TensorFlow backend.


In [2]:
N_SAMPLE = 25
TEST_SAMPLE_ID = 0

In [3]:
#Time-samples
#for all hdf5 files
all_hdf5_data = np.zeros((N_SAMPLE,12,650,650),dtype=np.uint16)


for i in range(N_SAMPLE):
    data_file = h5py.File(str(i) + '_level2a.h5', 'r')
    arr = data_file.get('bands')
   
    np_arr =  np.array(arr, dtype=np.uint16)
    
    all_hdf5_data[i] = np_arr
    

print(all_hdf5_data.shape)

#print(all_hdf5_data[9])


(25, 12, 650, 650)


In [4]:
# labels 
# actually cholorophil-a values for regression
import pandas
import sys 

labels_index = np.zeros((N_SAMPLE * 10), dtype=np.float16)

labels_name = np.zeros((N_SAMPLE * 10), dtype=np.float16)

df = pandas.read_csv('chl_values.csv')
    #print(df['Label'])
labels_name = df['Chl_Value']
#print(type(labels_name))

for index in range(len(labels_name)):
    #print(labels_name[index])
    labels_index[index] = labels_name[index]; 
    #print(index, labels_index[index])


print(len(labels_index))


250


In [5]:
#coordinate
s = pandas.read_csv('sensor_pos.csv')
#print(df['Label'])
S_POS_X = s['X']
S_POS_Y = s['Y']
#print(S_POS_X)

In [6]:
#patches from hdf5 file
#convert to numpy array
#n1 = np.array(n1, dtype=np.uint16)
n1 = np.zeros((12, 1), dtype= np.uint16)
#print (n1.shape)

patches_from_sensors = np.zeros((10*N_SAMPLE, 12 , 1), dtype = np.uint16)
#print(patches_from_sensors.shape, patches_from_sensors.dtype)
#print(patches_from_sensors[0])
new_feature_time = np.zeros((1), dtype=np.uint16)
# Sensor position (X, Y)
# 3X3 Patches 
# extract patch from data, starts (X-1,Y-1) ends ( X+1, Y+1)
# DO NOT FORGET!!!! Meaning of [x:y] is that the new array includes values at the X pos., but not at the Y pos.

# Sensor positions
#X = 10
#Y = 11
STEPS = 1

#patch = n1[:, X - STEPS: X + STEPS + 1, Y - STEPS: Y + STEPS + 1]
#print(patch.shape, patch)

#patches_from_sensors[0] = patch
#print(patches_from_sensors[0])
total = 0


    #n1 = pca_hdf5_data[i_sample] # After PCA data
    #n1 = ch0_hdf5_data[i_sample] # 1 channel data
    #print(i_sample,"Sample : ", n1.shape) 
    #extract 10 (# stations) patches for one sample(for one hdf5 file)
for i_s in range(len(S_POS_X)):
        X = S_POS_X[i_s]
        Y = S_POS_Y[i_s]
        for i_sample in range(N_SAMPLE):
            n1 = all_hdf5_data[i_sample] #original data
        # print("X - Y:" , X , Y )
        # Patch Shapes : [C,W,H]
        #patch = n1[:, X - STEPS: X + STEPS + 1, Y - STEPS: Y + STEPS + 1] #for patch
        #print(i_s,"s: ", patch.shape)
        #for a pixel
        # If you would like to use more than one channel
        # You should change like patch = n1[ :, X: X+ 1, Y: Y + 1] 
            patch = n1[:, X: X+ 1, Y: Y + 1] 
        
        #print(patch)
                
        #patch = np.reshape(patch,(patch.shape[0],1))
        #patch = np.append(patch, new_feature_time) #for time
        #print("Patch "  ,patch.shape)
        #patch[12] = i_s
            patch = np.reshape(patch,(patch.shape[0],1))
        #patnp.append([[1, 2, 3], [4, 5, 6]], [[7, 8, 9]], axis=0)
            patches_from_sensors[i_sample + (25 * i_s)] = patch
        
    
print("Number of all samples : ",len(patches_from_sensors))
print(" shape: ",patches_from_sensors.shape)

('Number of all samples : ', 250)
(' shape: ', (250, 12, 1))


In [7]:
import os
def write_txt(data,label, file_n):
    with open(file_n, 'a') as f1:
        for index in range(data.shape[0]):
            for index_i in range(12):
               #f1.write(data[index][index_i])
                f1.write('{: }'.format(data[index][index_i]))
                f1.write(',')
                
            f1.write('{: }'.format(label[index]))
           
            f1.write('\n')

    f1.close()
    
    
import os
def write_txt_label(data, file_n):
    with open(file_n, 'a') as f1:
        for index in range(data.shape[0]):
            f1.write('{: }'.format(data[index]))
            f1.write(' ')
            
            f1.write('\n')

    f1.close()

In [ ]:
# train test split
train_start_pos = 0*26
train_end_pos = 8*26
test_end_pos = 10*26

i = 9
X_train = np.vstack((patches_from_sensors[0*26:i*26], patches_from_sensors[(i+1)*26:10*26,:,:]) )

y_train = np.append(labels_index[0*26:i*26], labels_index[(i+1)*26:10*26] )

X_train = np.reshape(X_train,  (X_train.shape[0], 12))
#y_train = np.reshape(y_train, (y_train.shape[0], 1))


print("X_train", X_train.shape)
print("y_train", y_train.shape)

X_test = patches_from_sensors[i*26:(i+1)*26,:,:]
#X_test = np.append(X_test, patches_from_sensors[9:10,:,:])
y_test =labels_index[i*26:(i+1)*26]

X_test = np.reshape(X_test, (X_test.shape[0], 12))
#y_test = np.reshape(y_test, (y_test.shape[0], 1))

print("X_test", X_test.shape)
print("y_test", y_test)


write_txt(X_train, y_train, 'train_' + str(i) + '.txt')

write_txt(X_test, y_test , 'test_' + str(i) + '.txt')



In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

for i in range(10):
    X_train = np.vstack((patches_from_sensors[0*N_SAMPLE:i*N_SAMPLE], patches_from_sensors[(i+1)*N_SAMPLE:10*N_SAMPLE,:,:]) )

    y_train = np.append(labels_index[0*N_SAMPLE:i*N_SAMPLE], labels_index[(i+1)*N_SAMPLE:10*N_SAMPLE] )

    X_train = np.reshape(X_train,  (X_train.shape[0], 12))
    #y_train = np.reshape(y_train, (y_train.shape[0], 1))


    #print("X_train", X_train.shape)
    #print("y_train", y_train.shape)

    X_test = patches_from_sensors[i*N_SAMPLE:(i+1)*N_SAMPLE,:,:]
    #X_test = np.append(X_test, patches_from_sensors[9:10,:,:])
    y_test =labels_index[i*N_SAMPLE:(i+1)*N_SAMPLE]

    X_test = np.reshape(X_test, (X_test.shape[0], 12))
    #y_test = np.reshape(y_test, (y_test.shape[0], 1))

    #print("X_test", X_test.shape)
    #print("y_test", y_test)
    
    
    regr = RandomForestRegressor(n_estimators = 100, random_state=42).fit(X_train, y_train)
    print("----------------RF REG---------------------")
    print("---------------TEST - "  + str(i) +  "--------------------")
    # Make predictions using the testing set
    y_pred = regr.predict(X_test) #for SVR

    #print(y_pred)
    #print("Real Values : ",y_test)
    #print("Predictions : ", y_pred) 
    r = np.corrcoef(y_test, y_pred)

    # Correlation Coefficients
    print('Correlation Coef.: ',r[0, 1] )
    # The mean squared error
    print('Mean squared error: %.2f'
          % mean_squared_error(y_test, y_pred))
    # The coefficient of determination: 1 is perfect prediction
    print('Coefficient of determination: %.2f'
          % r2_score(y_test, y_pred))
    print('Mean absolute error: %.2f'
          % mean_absolute_error(y_test, y_pred))
 


    write_txt(X_train, y_train, 'train_' + str(i) + '.arff')

    write_txt(X_test, y_test , 'test_' + str(i) + '.arff')

----------------RF REG---------------------
---------------TEST - 0--------------------
('Correlation Coef.: ', 0.3524053014570755)
Mean squared error: 590.81
Coefficient of determination: 0.12
Mean absolute error: 20.56
----------------RF REG---------------------
---------------TEST - 1--------------------
('Correlation Coef.: ', 0.7577057964813403)
Mean squared error: 426.29
Coefficient of determination: 0.30
Mean absolute error: 15.31
----------------RF REG---------------------
---------------TEST - 2--------------------
('Correlation Coef.: ', 0.8436827197292484)
Mean squared error: 285.73
Coefficient of determination: 0.51
Mean absolute error: 12.45
----------------RF REG---------------------
---------------TEST - 3--------------------
('Correlation Coef.: ', 0.7732962747351083)
Mean squared error: 127.13
Coefficient of determination: 0.59
Mean absolute error: 7.93
----------------RF REG---------------------
---------------TEST - 4--------------------
('Correlation Coef.: ', 0.069

In [ ]:
# Plot outputs
plt.scatter(X_test[:,0], y_test,  color='black')
plt.plot(X_test, y_pred, color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()


In [ ]:
#SUPPORT VECTOR REG.
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
from sklearn import svm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import BayesianRidge
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor

# reshape data
X_train = np.reshape(X_train, (X_train.shape[0], 12))

X_test = np.reshape(X_test, (X_test.shape[0], 12))


# Fit regression model
#svr_rbf = svm.SVR(kernel='sigmoid', C=100, gamma=0.1, epsilon=.1).fit(X_train, y_train)





y_test = np.reshape(y_test, (y_test.shape[0]))

# Train the model using the training sets

#regr = svm.SVR().fit(X_train, y_train)
#regr =  BayesianRidge(compute_score=True).fit(X_train, y_train)
#regr = tree.DecisionTreeRegressor().fit(X_train, y_train)
regr = RandomForestRegressor(n_estimators = 100, random_state=42).fit(X_train, y_train)
print("----------------RF REG---------------------")

# Make predictions using the testing set
y_pred = regr.predict(X_test) #for SVR

#print(y_pred)
print("Real Values : ",y_test)
print("Predictions : ", y_pred) 
#r = np.corrcoef(y_test, y_pred)

# Correlation Coefficients
#print('Correlation Coef.: ',r[0, 1] )
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, y_pred))
print('Mean absolute error: %.2f'
      % mean_absolute_error(y_test, y_pred))
# Plot outputs
plt.scatter(X_test[:,0], y_test,  color='black')
plt.plot(X_test, y_pred, color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()





In [ ]:
#SUPPORT VECTOR REG.
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
from sklearn import svm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import BayesianRidge
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor

# reshape data
X_train = np.reshape(X_train, (X_train.shape[0], 13))

X_test = np.reshape(X_test, (X_test.shape[0], 13))

svr_poly = svm.SVR(kernel='poly', C=100, gamma=0.1, degree=3, epsilon=.1,
               coef0=1).fit(X_train, y_train)


print("----------------SVR-POLY---------------------")
# Make predictions using the testing set
y_pred = svr_poly.predict(X_test) #for SVR


#print(y_pred)
print("Real Values : ",y_test)
print("Predictions : ", y_pred) 


print('Correlation Coef.: %.2f'
      % np.corrcoef(y_test, y_pred))
# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, y_pred))
print('Mean absolute error: %.2f'
      % mean_absolute_error(y_test, y_pred))
# Plot outputs
plt.scatter(X_test[:,0], y_test,  color='black')
plt.plot(X_test, y_pred, color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()


In [ ]:
#SUPPORT VECTOR REG.
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
from sklearn import svm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import BayesianRidge
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor

# reshape data
X_train = np.reshape(X_train, (X_train.shape[0], 13))

X_test = np.reshape(X_test, (X_test.shape[0], 13))

svr_lin = svm.SVR(kernel='linear', C=100, gamma='auto').fit(X_train, y_train)
print("----------------SVR-LINEAR---------------------")

# Make predictions using the testing set
y_pred = svr_lin.predict(X_test) #for SVR



#print(y_pred)
print("Real Values : ",y_test)
print("Predictions : ", y_pred) 

# The mean squared error
print('Mean squared error: %.2f'
      % mean_squared_error(y_test, y_pred))
# The coefficient of determination: 1 is perfect prediction
print('Coefficient of determination: %.2f'
      % r2_score(y_test, y_pred))
print('Mean absolute error: %.2f'
      % mean_absolute_error(y_test, y_pred))
# Plot outputs
plt.scatter(X_test[:,0], y_test,  color='black')
plt.plot(X_test, y_pred, color='blue', linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()